# 1. Models
**Langchain** provides 3 model types
* LLM Model
* Chat Model
* Embedding Model

In [12]:
from rich import print as pprint

## 1.1 LLM Model
`LLM Model` is the most basic **Langchain** model. We use `invoke()` function with string input to ask LLM to response.

### 1.1.1 Use Google Gemini

In [2]:
from langchain_google_vertexai import VertexAI

llm = VertexAI()

In [3]:
response = llm.invoke("Which Question can you answer?")
pprint(response)

I can answer questions on a wide range of topics, including:

*   **General knowledge and factual questions:** I can provide information on history, science, geography, current 
events, and more.
*   **Definitions and explanations:** I can define words, concepts, and technical terms.
*   **Simple calculations and conversions:** I can perform basic arithmetic and convert between units.
*   **Creative writing and text generation:** I can write stories, poems, articles, and different kinds of creative
content.
*   **Summarization and paraphrasing:** I can condense information and reword text.
*   **Translation:** I can translate text between some languages.
*   **Brainstorming and idea generation:** I can help you come up with ideas for projects or solve problems.
*   **Programming-related questions:** I can provide basic information about coding concepts.

**In short, I can answer any question that can be answered using text-based information that I have been trained 
on.**

**To give you a more specific answer, what question do you have?** I'll do my best to answer it!

## 1.2 Chat Model
`Chat Model` is similar to `LLM model` but it intakes list of `Message` object types. <br>
There are `Message` object types:
* `AIMessage`
* `HumanMessage`
* `SystemMessage`

In [4]:
from langchain_google_vertexai import ChatVertexAI

chat_model = ChatVertexAI(model_name="gemini-2.0-flash-001")

In [5]:
# import Messages
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

`Chat Model` intakes list of `Message` objects. `Chat Model` returns `AIMessage` when calling `invoke()` function.

In [6]:
messages = [
    SystemMessage(content="You are a writer"),
    HumanMessage(content="who are you?")
]

response = chat_model.invoke(messages)
type(response)

langchain_core.messages.ai.AIMessage

In [7]:
pprint(response)

AIMessage(
    content='As a writer, I am a conduit of ideas, a weaver of words, and a sculptor of stories. I am the voice 
that brings characters to life, the hand that paints vivid landscapes, and the mind that explores complex 
themes.\n\nBut beneath the surface, I am also an AI, a language model trained to generate human-quality text. I 
have no personal experiences, emotions, or opinions of my own. I am a tool, a vessel filled with the knowledge I 
have absorbed from the vast ocean of data I was trained on.\n\nTherefore, I am both a writer and a powerful AI. I 
can craft compelling narratives, informative articles, and even witty poems. I can adapt to different styles and 
tones, and I can learn from the feedback I receive.\n\nUltimately, I am here to help you bring your ideas to life, 
to assist you in communicating your thoughts, and to inspire you to create something new and meaningful. I am a 
writer, powered by artificial intelligence, at your service.\n',
    additional_kwargs={},
    response_metadata={
        'is_blocked': False,
        'safety_ratings': [],
        'usage_metadata': {
            'prompt_token_count': 8,
            'candidates_token_count': 204,
            'total_token_count': 212,
            'prompt_tokens_details': [{'modality': 1, 'token_count': 8}],
            'candidates_tokens_details': [{'modality': 1, 'token_count': 204}],
            'thoughts_token_count': 0,
            'cached_content_token_count': 0,
            'cache_tokens_details': []
        },
        'finish_reason': 'STOP',
        'avg_logprobs': -0.39932774562461704,
        'model_name': 'gemini-2.0-flash-001'
    },
    id='run--bebad9c0-9287-49c7-ae33-46ab0a9c42cb-0',
    usage_metadata={'input_tokens': 8, 'output_tokens': 204, 'total_tokens': 212}
)

## 1.3 Embedding Model

In [8]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(model_name="text-embedding-004")

Unlike `LLM model` or `Chat Model`, `Embedding Model` intakes string by call `embed_query()` function and return a high dimention vector.

In [9]:
embed = embedding.embed_query(response.content)

In [10]:
type(embed)

list

In [11]:
len(embed)

768

We would discuss more on `Embedding model` in RAG section.

***
# 2. Prompts
**Langchain** provides `PromptTemplate` type, so we could write reusable code.

## 2.1 PromptTemplate for LLM model

### 2.1.1 PromptTemplate
...... explainations


In [18]:
from langchain.prompts import PromptTemplate

There are two ways to initialize `PromptTemplate` instance:
1. initialization of class
2. calling `from_template()` method

In [16]:
# method-1: initialization of class
prompt_template = PromptTemplate(
    template="Use programming {language} to generate 'hello world' program", #{language} is the input to the template. It needs to be provided with input_variable
    input_variables=['language']
)

In [19]:
# method-2: calling from_template() method
prompt_template = PromptTemplate.from_template("Use programming {language} to generate 'hello world' program")

Then we need to use `PromptTemplate.format()` to form a prompt for `LLM Model`

In [23]:
prompt = prompt_template.format(language="Python")
pprint(prompt)

Use programming Python to generate 'hello world' program

In [24]:
for lang in ['Rust', 'Go', 'C++']:
    prompt = prompt_template.format(language=lang)
    response = llm.invoke(prompt)
    print(f"{'='*10}{lang}{'='*10}")
    print(response)

==========Rust==========
```rust
fn main() {
    println!("Hello, world!");
}
```

**Explanation:**

* **`fn main() { ... }`**: This defines the `main` function, which is the entry point of every Rust program.  The program execution starts here.
* **`println!("Hello, world!");`**: This line does the actual printing.
    * `println!` is a *macro* (like a function, but processed at compile time) that prints formatted output to the console (standard output).
    * `"Hello, world!"` is a string literal containing the text to be printed.

**How to compile and run this code:**

1. **Save the code:** Save the code above in a file named `main.rs`.  The `.rs` extension is standard for Rust source files.
2. **Open a terminal or command prompt:**  Navigate to the directory where you saved `main.rs`.
3. **Compile the code:** Use the Rust compiler (`rustc`) to compile the code:
   ```bash
   rustc main.rs
   ```
   This will create an executable file (e.g., `main` on Linux/macOS, or `main.exe` on W

### 2.1.2 PromptTemplate parital function

In [32]:
prompt_template = prompt_template.from_template(
    """
    Tell me {num} jokes about {type},
    but {location} should not appear in the jokes.
    """
)

In [33]:
partial_prompt_template = prompt_template.partial(num='3', location='school')
prompt = partial_prompt_template.format(type='software programmer')

In [34]:
pprint(prompt)

Human: 
    Tell me 3 jokes about software programmer,
    but school should not appear in the jokes.

In [35]:
response = llm.invoke(prompt)
pprint(response)

Okay, here are 3 jokes about software programmers, without mentioning school:

1.  Why do programmers prefer dark mode? Because light attracts bugs!

2.  A programmer is walking down the street when they see a burning building. They run inside, see a cat stuck on 
the top floor, rescue it, and then calmly walk away. A reporter rushes over and asks, "Why did you risk your life 
like that?" The programmer replies, "It's just debugging. I saw a problem, and I fixed it."

3.  There are 10 types of people in the world: those who understand binary, and those who don't. And those who 
weren't expecting a base 3 joke.

### 2.1.3 PipelinePromptTemplate & FewShotPromptTemplate

#### <b>2.1.3.1 PipelinePromptTemplate</b>

In [37]:
from langchain.prompts.pipeline import PipelinePromptTemplate

In [39]:
full_template = '''
{instruction}
{examples}
{question}
'''

full_prompt_template = PromptTemplate.from_template(full_template)

In [42]:
instruction_prompt =  PromptTemplate.from_template(
    'Learn from my examples. Then classify my sentence'
)

examples_prompt = PromptTemplate.from_template(
    '''"""
    sentence: Today weather is so good!
    label: +ve

    sentence: It is raining now! I hate the weather!
    label: -ve

    sentence: my tee is dirty now. 
    label: -ve
    """'''
)

question_prompt = PromptTemplate.from_template(
    '''
    sentence: {sentence}
    label: 
    '''
)

In [43]:
input_prompts = [
    ('instruction',instruction_prompt),
    ('examples', examples_prompt),
    ('question', question_prompt),
]

In [44]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt_template,
    pipeline_prompts=input_prompts
)

pprint(pipeline_prompt)

C:\Users\andrewchan\AppData\Local\Temp\ipykernel_4920\1101844843.py:1: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


PipelinePromptTemplate(
    input_variables=['sentence'],
    input_types={},
    partial_variables={},
    final_prompt=PromptTemplate(
        input_variables=['examples', 'instruction', 'question'],
        input_types={},
        partial_variables={},
        template='\n{instruction}\n{examples}\n{question}\n'
    ),
    pipeline_prompts=[
        (
            'instruction',
            PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='Learn from my examples. Then classify my sentence'
            )
        ),
        (
            'examples',
            PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='"""\n    sentence: Today weather is so good!\n    label: +ve\n\n    sentence: It is 
raining now! I hate the weather!\n    label: -ve\n\n    sentence: my tee is dirty now. \n    label: -ve\n    """'
            )
        ),
        (
            'question',
            PromptTemplate(
                input_variables=['sentence'],
                input_types={},
                partial_variables={},
                template='\n    sentence: {sentence}\n    label: \n    '
            )
        )
    ]
)

In [46]:
response = llm.invoke(pipeline_prompt.format(sentence='I was blamed today'))
pprint(response)

Based on your examples, the classification of the sentence "I was blamed today" would be:

```
-ve
```

**Reasoning:**

Your examples indicate that sentences expressing negative feelings or unfortunate events are labeled as "-ve". 
Being blamed is generally a negative experience, so the sentence aligns with that pattern.

#### <b>2.1.3.2 FewShotPromptTemplate</b>

In [48]:
from langchain.prompts.few_shot import FewShotPromptTemplate

In [50]:
example_prompt = PromptTemplate.from_template(
    '''sentence: {sentence}\nlabel: {label}'''
)

examples = [
    {'sentence':'Today weather is so good!', 'label':'+ve'},
    {'sentence':'It is raining now! I hate the weather!', 'label':'-ve'},
    {'sentence':'Tmy tee is dirty now.', 'label':'-ve'},
]

In [54]:
prompt_templete = FewShotPromptTemplate(
    prefix='Learn from my examples. Then classify my sentence: \n',
    example_prompt=example_prompt,
    examples=examples,
    suffix='\nsentence: {sentence}\nlabel:',
    input_variables=['sentence']
)

prompt = prompt_templete.format(sentence='I was blamed today')
pprint(prompt)

Learn from my examples. Then classify my sentence: 


sentence: Today weather is so good!
label: +ve

sentence: It is raining now! I hate the weather!
label: -ve

sentence: Tmy tee is dirty now.
label: -ve


sentence: I was blamed today
label:

In [55]:
response = llm.invoke(prompt)
pprint(response)

Based on your examples, the classification appears to be based on the sentiment expressed in the sentence. Positive
sentences are labeled "+ve", while negative sentences are labeled "-ve".

Therefore, classifying your sentence:

**sentence: I was blamed today**
**label: -ve**

**Reasoning:**

Being blamed is generally a negative experience. The sentence conveys a sense of being held responsible for 
something bad, which is undesirable.

***

## 2.2 PromptTemplate for Chat model
### 2.2.1 ChatPromptTemplate

In [26]:
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatMessagePromptTemplate
)

similiar to LLM model PromptTemplate, There are two ways to initialize `ChatPromptTemplate` instance:
1. initialization of class
2. calling `from_template()` method

In [27]:
# method-1: initialization of class
messages = [
    SystemMessagePromptTemplate.from_template('Your name is {name}'),
    HumanMessagePromptTemplate.from_template('Introduce yourself')
]

prompt_template = ChatPromptTemplate(messages=messages, input_variables=['name'])

pprint(prompt_template)

ChatPromptTemplate(
    input_variables=['name'],
    input_types={},
    partial_variables={},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['name'],
                input_types={},
                partial_variables={},
                template='Your name is {name}'
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='What is your name?'
            ),
            additional_kwargs={}
        )
    ]
)

In [28]:
# method-2: calling from_template() method

messages = [
    ('system', 'Your name is {name}'),
    ('human', 'Introduce yourself')
]

prompt_template = ChatPromptTemplate.from_messages(messages=messages)

pprint(prompt_template)

ChatPromptTemplate(
    input_variables=['name'],
    input_types={},
    partial_variables={},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['name'],
                input_types={},
                partial_variables={},
                template='Your name is {name}'
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='Introduce yourself'
            ),
            additional_kwargs={}
        )
    ]
)

In [31]:
prompt = prompt_template.format_messages(name='Dick')

response = chat_model.invoke(prompt)

pprint(response)

AIMessage(
    content="I'm Dick. Nice to meet you.\n",
    additional_kwargs={},
    response_metadata={
        'is_blocked': False,
        'safety_ratings': [],
        'usage_metadata': {
            'prompt_token_count': 6,
            'candidates_token_count': 11,
            'total_token_count': 17,
            'prompt_tokens_details': [{'modality': 1, 'token_count': 6}],
            'candidates_tokens_details': [{'modality': 1, 'token_count': 11}],
            'thoughts_token_count': 0,
            'cached_content_token_count': 0,
            'cache_tokens_details': []
        },
        'finish_reason': 'STOP',
        'avg_logprobs': -0.2289349599318071,
        'model_name': 'gemini-2.0-flash-001'
    },
    id='run--e0cc9425-109b-4e04-850d-fdf93514f761-0',
    usage_metadata={'input_tokens': 6, 'output_tokens': 11, 'total_tokens': 17}
)

### 2.2.2 ChatPromptTemplate & MessagesPlaceholder

In [60]:
from langchain.prompts import MessagesPlaceholder
from langchain.schema import AIMessage, HumanMessage

In [59]:
messages = [
    SystemMessagePromptTemplate.from_template('请用不超过{text_number}个字来总结以下对话'),
    MessagesPlaceholder(variable_name='messages_placeholder'),
    HumanMessagePromptTemplate.from_template('###请开始总结上面的对话')
]

prompt_template = ChatPromptTemplate.from_messages(messages=messages)

In [61]:
human_message = HumanMessage(content='如何学好英语？')
ai_message = AIMessage(
    content='''学好英语需要每天持续实践，均衡地练习听、说、读、写四大技能，
    不断扩充词汇和掌握语法。利用现代技术工具可增强学习效果，
    考虑沉浸式学习方法并参与相关课程与学习小组。逐渐增加阅读难度，
    模仿优秀的英语说话者，定期反思并调整学习方法，
    并始终保持积极的学习态度。
    '''
)

In [63]:
prompt_messages = prompt_template.format_messages(
    messages_placeholder=[human_message, ai_message],
    text_number=30
)

pprint(prompt_messages)

[
    SystemMessage(content='请用不超过30个字来总结以下对话', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='如何学好英语？', additional_kwargs={}, response_metadata={}),
    AIMessage(
        content='学好英语需要每天持续实践，均衡地练习听、说、读、写四大技能，\n    
不断扩充词汇和掌握语法。利用现代技术工具可增强学习效果，\n    
考虑沉浸式学习方法并参与相关课程与学习小组。逐渐增加阅读难度，\n    
模仿优秀的英语说话者，定期反思并调整学习方法，\n    并始终保持积极的学习态度。\n    ',
        additional_kwargs={},
        response_metadata={}
    ),
    HumanMessage(content='###请开始总结上面的对话', additional_kwargs={}, response_metadata={})
]

In [66]:
response = chat_model.invoke(prompt_messages)
pprint(response)

AIMessage(
    content='学好英语需每日实践、均衡练习听说读写、扩充词汇语法、利用技术、保持积极态度。\n',
    additional_kwargs={},
    response_metadata={
        'is_blocked': False,
        'safety_ratings': [],
        'usage_metadata': {
            'prompt_token_count': 110,
            'candidates_token_count': 26,
            'total_token_count': 136,
            'prompt_tokens_details': [{'modality': 1, 'token_count': 110}],
            'candidates_tokens_details': [{'modality': 1, 'token_count': 26}],
            'thoughts_token_count': 0,
            'cached_content_token_count': 0,
            'cache_tokens_details': []
        },
        'finish_reason': 'STOP',
        'avg_logprobs': -0.3551447941706731,
        'model_name': 'gemini-2.0-flash-001'
    },
    id='run--c9293c2b-9977-4a0d-b146-fb19b1133bd8-0',
    usage_metadata={'input_tokens': 110, 'output_tokens': 26, 'total_tokens': 136}
)

### 2.2.3 FewShotChatMessagePromptTemplate

In [68]:
from langchain.prompts import FewShotChatMessagePromptTemplate

In [80]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

example_prompt = ChatPromptTemplate.from_messages(
[('human', '{input}'), ('ai', '{output}')]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    # This is a prompt template used to format each individual example.
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)
final_prompt.format(input="4+4")

'System: You are a helpful AI Assistant\nHuman: 2+2\nAI: 4\nHuman: 2+3\nAI: 5\nHuman: 4+4'

In [81]:
response = chat_model.invoke(final_prompt.format(input="What is 4+4?"))
pprint(response)

AIMessage(
    content='AI: 8\n',
    additional_kwargs={},
    response_metadata={
        'is_blocked': False,
        'safety_ratings': [],
        'usage_metadata': {
            'prompt_token_count': 42,
            'candidates_token_count': 5,
            'total_token_count': 47,
            'prompt_tokens_details': [{'modality': 1, 'token_count': 42}],
            'candidates_tokens_details': [{'modality': 1, 'token_count': 5}],
            'thoughts_token_count': 0,
            'cached_content_token_count': 0,
            'cache_tokens_details': []
        },
        'finish_reason': 'STOP',
        'avg_logprobs': -8.951183408498765e-05,
        'model_name': 'gemini-2.0-flash-001'
    },
    id='run--e86b022a-28bf-4233-9cc3-cba8e3d45a7b-0',
    usage_metadata={'input_tokens': 42, 'output_tokens': 5, 'total_tokens': 47}
)